## Exemplo de aplicação: compressão de imagens utilizando decomposição em valores singulares (SVD)

Este arquivo .ipynb deve ser utilizado para realizar a Parte II da atividade sobre autovalores e autovetores da disciplina de Cálculo Avançado. O objetivo aqui consiste em utilizar SVD - uma técnica que se baseia na identificação dos autovalores e autovetores de uma matriz- para comprimir imagens. Mais informações sobre SVD podem ser conferidas no pdf fornecido junto com este arquivo.

In [ ]:
#Importação das bibliotecas de interesse:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from PIL import Image

In [ ]:
# Vamos especificar o nome do arquivo cuja imagem iremos ler, bem como o formato utilizado (no caso, jpg) e o local onde o arquivo está salvo (modifique de acordo com
# o local onde você vá salvar o arquivo na sua máquina). 

nome_do_arquivo = '' #Você deve alterar o nome conforme o último algarismo do seu RA.
formato = '.jpg'
local = "" #Obs.: barras invertidas (\) devem ser substituídas por barras (/) ao especificar o local.

# Representação matricial da imagem selecionada

Usando o módulo matplotlib.image, vamos ler o arquivo .jpg, e representá-la matricialmente:

In [ ]:
# Cada célula da matriz corresponde a um pixel da imagem, em escala de cinza (preto corresponde ao valor 0, branco corresponde
# ao valor de 255). A seguir, estamos imprimindo a matriz:
X = imread(local+nome_do_arquivo+formato)
print("X = ", np.matrix(X))

Vamos visualizar aqui a imagem sendo carregada:

In [ ]:
plt.rcParams['figure.figsize'] = [16, 8]
plt.figure()
img = plt.imshow(X)
img.set_cmap('gray')
plt.axis('off')
plt.show()

## Decomposição em Valores Singulares:

Vamos agora calcular a decomposição da matriz X em valores singulares (SVD). As matrizes U, S e V correspondem às matrizes de mesmo nome explicadas nas instruções referentes a esta atividade.

In [ ]:
U, S, V = np.linalg.svd(X,full_matrices=False)
S = np.diag(S)

Note que a matriz S é diagonal (ou seja, todos os elementos são iguais a zero, exceto aqueles na diagonal principal):

In [ ]:
print(np.matrix(S))

Vamos ver o número total de valores singulares em S:

In [ ]:
numero_total_de_valores_singulares = len(S)

A diagonal principal de S é dada por:

In [ ]:
print(np.diag(S))

Note que os valores singulares estão ordenados do maior para o menor (o método linalg.svd já realiza essa ordenação). Vamos plotar em um gráfico os valores singulares associados à matriz X (note que o eixo y está em escala logarítmica)

In [ ]:
plt.figure(1)
plt.semilogy(np.diag(S))
plt.title('Valores Singulares')
plt.grid()
plt.show()
plt.savefig(local+'valores_singulares.jpg', format='jpg')

Vamos imprimir na tela o maior e o menor valor singular contidos em S:

In [ ]:
print("O maior valor singular em S é: ",max(np.diag(S))) 
print("O menor valor singular em S é: ",min(np.diag(S))) 

Note que a diferença entre o maior e o menor valor singular é muito significativa. Quanto maior for um valor singular, mais importante é o autovetor correspondente a ele para a formação da imagem. Isto significa que podemos nos livrar dos autovetores associados aos menores valores singulares, sem que isso represente uma perda muito substancial na qualidade. Para verificar isso graficamente, vamos plotar a porcentagem que os i maiores valores singulares (i indo de 0 até o número de elementos em S), quando somados, representam em relação ao total de valores singulares:

In [ ]:
eixo_x = 100*np.arange(1,len(np.diag(S))+1)/len(np.diag(S))
plt.figure(2)
plt.plot(eixo_x,100*np.cumsum(np.diag(S))/np.sum(np.diag(S)))
plt.title('Soma Acumulada (%)')
plt.xlabel('Porcentagem de Valores Singulares Mantidos')
plt.grid()
plt.show()
plt.savefig(local+'soma_acumulada.jpg', format='jpg')

Note que, dependendo da imagem, um único valor singular (aquele com índice 0, ou seja, na primeira linha e primeira coluna de S) pode representar, sozinho, 20% da soma de todos os valores singulares da imagem juntos (ou até mais)!

# Compressão

Utilizando o estudo realizado na seção anterior, vamos agora efetivamente comprimir a imagem utilizando SVD. A ideia é descartar os menores valores singulares, junto com os autovetores correspondentes, de modo a armazenar menos informação.

Para averiguar o efeito da compressão de forma gradual, vamos manter uma certa porcentagem de valores singulares e descartar o restante. Iremos começar com uma porcentagem de 90% (ou seja, estamos mantendo 90% dos autovetores e valores singulares da imagem original e descartando 10%), e ir reduzindo gradualmente até chegar em uma porcentagem de 5 % (ou seja, estamos descartando 95% dos autovetores e valores singulares da imagem original). Para isso, vamos criar um array de porcentagens, arranjadas em ordem decrescente:

In [ ]:
porcentagem_minima = 5
porcentagem_maxima = 90
passo = -1
porcentagens = np.arange(porcentagem_maxima,porcentagem_minima-1,passo)

Imprimindo na tela as porcentagens consideradas:

In [ ]:
print("Porcentagens consideradas: \n", porcentagens)

Agora, iremos realizar a compressão propriamente dita considerando cada porcentagem contida no vetor "porcentagens". Além disso, iremos salvar a imagem resultante no mesmo local em que a imagem original foi salva. Cada imagem comprimida será salva com o mesmo nome da imagem original, acrescido do sufixo "_comprimida_p", em que p é a porcentagem de valores singulares mantidos (com um zero à esquerda, se necessário).

In [ ]:
#Para cada elemento p do vetor porcentagens:
for p in porcentagens:
    # Vamos calcular o número r de valores singulares a serem considerados:
    r = math.ceil(numero_total_de_valores_singulares*p/100)
    # Vamos agora, reconstruir uma aproximação para X. Nós vamos selecionar apenas as primeiras r colunas de U e as primeiras 
    # r linhas de V (bem como as linhas e colunas correspondentes de S).
    X_aproximado = U[:,:r] @ S[0:r,:r] @ V[:r,:]
    # Em seguida, vamos converter todos os elementos da matriz X_aproximado para inteiros:
    X_aproximado = X_aproximado.astype(int)
    # O código a seguir servirá para imprimir na tela a imagem correspondente à matriz X_aproximado:
    print("Porcentagem de valores singulares mantidos: ",p,"% (porcentagem de valores singulares descartados: ",100-p,"%).")
    plt.rcParams['figure.figsize'] = [16, 8]
    plt.figure()
    img = plt.imshow(X_aproximado)
    img.set_cmap('gray')
    plt.axis('off')
    plt.show()
    # Especificando o nome a ser utilizado para salvar a imagem correspondente à matriz X_aproximado:
    nome_para_salvar = local + nome_do_arquivo+"_comprimida_"+f'{p:02d}'+formato
    # Gerando a imagem proprimante dita e salvando-a:
    im = Image.fromarray(X_aproximado).convert('L')
    im.save(nome_para_salvar)

Uma vez executado todo o código até aqui, responda às perguntas da Parte II da atividade.